# Seasonal Migration Patterns of the American White Pelican (Pelecanus erythrorhynchos)

## Species description
The American White Pelican is one of North America's largest birds, measuring from 1.4-1.8 m long with a wingspan of up to 2.4 m (Boreal Birds, 2025). When I moved to Colorado from the tropics, I was shocked to see such an instantly recognizable and iconic shorebird here along the Front Range. As it turns out, eastern populations of the American White Pelican winter along the Gulf of Mexico and migrate north in the spring to breeding grounds in the northern United States and Canada, passing right through Colorado! My biggest question was "what on earth are they eating, so far from the ocean?" This bird is a non-selective fish eater, foraging from reservoirs and wetlands along its migration route. Instead of diving for fish like the more dramatic seabirds, they will drive fish towards shore in a coordinated flock and scoop up their prey in shallow waters with their enormous bills. 

## Data description
I used two different datasets to map the migration patterns of the American White Pelican. First, I downloaded species occurrence data from the Global Biodiversity Information Facility (GBIF) from the year 2024, to limit the size of the dataset. I then downloaded ecoregion data from ArcGIS FeatureService, using the known locations of species observations from the GBIF file to inform which ecoregions I needed. 
(GBIF data citation below)

## Methods description
To make a map of observations in each month, I needed to normalize the occurrence data by sampling effort in each ecoregion and each month. Because some ecoregions are more or less populated, the data is biased towards more observations in areas with more people to do the observing. Additionally, there may be some months where people collecting data, from citizen scientitsts to birders to professionals, are out collecting data more than other months. I found the average number of occurrences for each ecoregion, and the average number of occurrences for each month. I then divided the occurrences per month in each ecoregion value by these two averages to normalize for sampling effort. 

I then made a plot of the normalized occurrences for each month of the year, check it out!
<embed type="text/html? src="pelican_migration.html" width="600" height = "600">

## American White Pelican Migration Plot
In the pelican migration plot you can clearly see how most American White Pelicans spend the winter months in the southern United States and Mexico and the summer months in the central United States up to central Canada. One thing that surprised me is that they do pass through the part of the United States where I grew up, around New York City. In New York spent a lot of time in the outdoors around wetlands from the spring to fall, and am surprised that I never noticed one of these very distinctive birds. It is also interesting to note that the peak times for American White Pelican observations here in Colorado appear to be April and August/September. It makes sense that they were on my mind, as I probably just saw some flying around a couple of weeks ago!

## References
https://www.borealbirds.org/bird/american-white-pelican#:~:text=American%20White%20Pelicans%20are%20gregarious,slowly%20and%20gracefully%20in%20circles.

GBIF.org (21 October 2025) GBIF Occurrence Download https://doi.org/10.15468/dl.5dzzmx


## Code
The following is the code I used to generate this map. 

In [ ]:
# Code for downloading species data, ecoregion data, 
# and generating an interactive species migration map.
# Code written by Earth Analytics Education Program
# Modified by Maddy Rich, 10.27.2025

# Import useful packages
import time
import zipfile
from getpass import getpass
from glob import glob
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
import pygbif.occurrences as occ
import pygbif.species as species
import requests
import json
import calendar
import cartopy.crs as ccrs
import panel as pn
import hvplot.pandas

####-----------------------------------####
#### DO NOT MODIFY THIS CHUNK OF CODE! ####
####-----------------------------------####
# This code ASKS for your credentials 
# and saves it for the rest of the session.
# NEVER put your credentials into your code!!!!

# GBIF needs a username, password, and email 
# All 3 need to match the account
reset = False

# Request and store username
if (not ('GBIF_USER'  in os.environ)) or reset:
    os.environ['GBIF_USER'] = input('GBIF username:')

# Securely request and store password
if (not ('GBIF_PWD'  in os.environ)) or reset:
    os.environ['GBIF_PWD'] = getpass('GBIF password:')
    
# Request and store account email address
if (not ('GBIF_EMAIL'  in os.environ)) or reset:
    os.environ['GBIF_EMAIL'] = input('GBIF email:')

####-----------------------------------####
####  Download species data from GBIF  ####
####-----------------------------------####
# Pull the species backbone for the North American White Pelican (Pelecanus erythrorhynchos)
# This backbone is a dictionary
backbone = species.name_backbone(name= "Pelecanus erythrorhynchos")

# Pull the taxon key and assign it to the variable taxon_key
taxon_key = backbone['usageKey']


# Pull the species backbone for the North American White Pelican (Pelecanus erythrorhynchos)
# This backbone is a dictionary
backbone = species.name_backbone(name= "Pelecanus erythrorhynchos")

# Pull the taxon key and assign it to the variable taxon_key
taxon_key = backbone['usageKey']

# Give the download a descriptive name
pelican_csv_path = original_gbif_path.with_name("pelican_gbif_data.csv")
# Move file to descriptive path
original_gbif_path.rename(pelican_csv_path)

# Load the GBIF data
pelican_df = pd.read_csv(pelican_csv_path, delimiter='\t')

# Convert to GeoDataFrame
pelican_gdf = (
    gpd.GeoDataFrame(
        pelican_df, 
        geometry=gpd.points_from_xy(
            pelican_df.decimalLongitude, 
            pelican_df.decimalLatitude), 
        crs="EPSG:4326")
    # Select the desired columns
    #[[]]
)
pelican_gdf
# Check results
pelican_gdf.total_bounds

####----------------------------------------------------####
#### Download ecoregion data from ArcGIS FeatureService ####
####----------------------------------------------------####

# Merge the GBIF observations into a single geometry
gbif_union = pelican_gdf.geometry.union_all().envelope
gbif_union

# Convert geometry to geoJSON
gbif_geojson = gbif_union.__geo_interface__

gbif_geojson

# Construct ArcGIS-compatible JSON
arcgis_geom = json.dumps(dict(
    rings=gbif_geojson["coordinates"],
    spatialReference={"wkid": 4326}
))

# Prepare API request
eco_url = (
    "https://services5.arcgis.com/0AFsQflykfA9lXZn"
    "/ArcGIS/rest/services"
    "/WWF_Terrestrial_Ecoregions_Of_The_World_official_teow"
    "/FeatureServer/0/query")
eco_params = {
    "f": "geojson",
    "where": "1=1",
    "outFields": "eco_code,area_km2",
    "returnGeometry": "true",
    # Return polygons containing any GBIF observation
    "spatialRel": "esriSpatialRelIntersects",  
    "geometryType": "esriGeometryPolygon",
    # Override web Mercator server default
    "inSR": "4326",
    "outSR": "4326",
    # Must format geometry
    "geometry": arcgis_geom
}

# Submit API request

eco_params_esri = eco_params.copy()
eco_params_esri["f"] = "json"   # <- ESRI JSON
# (keep outSR=4326, returnGeometry=true, etc.)
resp = requests.get(eco_url, params=eco_params_esri)
resp.raise_for_status()
esri = resp.json()

import json
from shapely.geometry import shape, Polygon, MultiPolygon
from shapely.validation import make_valid
import geopandas as gpd

def esri_to_geojson_geom(esri_geom):
    """Convert ESRI JSON polygon/multipolygon to GeoJSON geometry."""
    if "rings" in esri_geom:
        # ESRI Polygon
        return {
            "type": "Polygon",
            "coordinates": esri_geom["rings"]
        }
    elif "paths" in esri_geom:
        # Convert ESRI polyline → GeoJSON polyline
        return {
            "type": "LineString",
            "coordinates": esri_geom["paths"][0]
        }
    else:
        # Unsupported geometry type
        return None


records = []
geoms = []

for feat in esri.get("features", []):
    props = feat.get("attributes", {}) or {}
    esri_geom = feat.get("geometry")
    if not esri_geom:
        continue
    gj = esri_to_geojson_geom(esri_geom)
    if gj is None:
        continue
    try:
        geom = shape(gj)
        # Fix invalid rings (open rings, self-intersections, etc.)
        if not geom.is_valid:
            geom = make_valid(geom)
        # Ensure polygonal (skip weird cases where make_valid splits into garbage)
        if geom.geom_type not in ("Polygon", "MultiPolygon"):
            continue
        geoms.append(geom)
        records.append(props)
    except Exception:
        continue
    
pelican_eco_gdf = gpd.GeoDataFrame(records, geometry=geoms, crs="EPSG:4326")

# Check the download
pelican_eco_gdf.head()

####--------------------------------####
#### Normalize species observations ####
####--------------------------------####

pelican_eco_gdf = pelican_eco_gdf.rename_axis('ecoregion') 

pelican_eco_df = (
    pelican_eco_gdf
    # Match the CRS of the GBIF data and the ecoregions
    .to_crs(pelican_gdf.crs)
    # Find ecoregion for each observation
    .sjoin(
        pelican_gdf,
        how='inner', 
        predicate='contains')
    # Select the required columns
    [['index_right', 'month']]
    .rename(columns={
        'index_right': 'observation_id'
    })
    )    

pelican_eco_df

pelican_occurrence_df = (
    pelican_eco_df
    # For each ecoregion, for each month...
    .groupby(['ecoregion', 'month'])
    # ...count the number of occurrences
    .agg(occurrences=('observation_id', 'count'))
)

# Get rid of rare observations (possible misidentification?)
# We only want rows where occurrences are greater than one
pelican_occurrence_df = pelican_occurrence_df[pelican_occurrence_df.occurrences > 1]

# Take the mean by ecoregion
mean_occurrences_by_ecoregion = (
    pelican_occurrence_df
    .groupby('ecoregion')
    .mean()
)
# Take the mean by month
mean_occurrences_by_month = (
    pelican_occurrence_df
    .groupby('month')
    .mean()
)

# Normalize by space and time for sampling effort

pelican_occurrence_df['norm_occurrences'] = (
    pelican_occurrence_df
    / mean_occurrences_by_ecoregion
    / mean_occurrences_by_month
)

####----------------------------------------####
#### Map normalized species occurrence data ####
####----------------------------------------####

# Simplify the geometry to speed up processing
pelican_eco_gdf.geometry = pelican_eco_gdf.simplify(.01, preserve_topology=False)

# Change the CRS to Mercator for mapping
pelican_eco_gdf = pelican_eco_gdf.to_crs(ccrs.Mercator())

# Check that the plot runs in a reasonable amount of time
# This did take 1 minute and 13 seconds on Maddy's machine
pelican_eco_gdf.hvplot(geo=True, crs=ccrs.Mercator())

# Rename the ecoregions index as "ecoregion" so the join below works
pelican_eco_gdf = pelican_eco_gdf.rename_axis('ecoregion')

# Join the occurrences with the plotting GeoDataFrame
pelican_occurrence_gdf = pelican_eco_gdf.join(pelican_occurrence_df[['norm_occurrences']])

# Get the plot bounds so they don't change with the slider
xmin, ymin, xmax, ymax = pelican_occurrence_gdf.total_bounds

# Plot occurrence by ecoregion and month
pelican_migration_plot = (
    pelican_occurrence_gdf
    .hvplot.polygons(
        c='norm_occurrences',
        groupby='month',
        # Get the slider set up
        widget_location='bottom',
        # Use background tiles
        geo=True, 
        crs=ccrs.PlateCarree(), 
        tiles='CartoLight',
        project=True,
        color='norm_occurrences',
        title="American White Pelican Migration",
        xlim=(xmin, xmax), ylim=(ymin, ymax),
        frame_height=600
    )
)

# Save the plot
pelican_migration_plot.save('pelican_migration.html', embed=True)

pelican_migration_plot